In [ ]:
import os, shutil
shutil.copytree('/kaggle/input/pytorch-yolov4', '/kaggle/working/yolov4')
os.chdir('/kaggle/working/yolov4')

In [ ]:
!pip install /kaggle/input/easydict/easydict-1.9-py2.py3-none-any.whl
!pip install /kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl

Processing /kaggle/input/easydict/easydict-1.9-py2.py3-none-any.whl
Processing /kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl


## Global parameters

In [ ]:
RUN_TEST = False  # Use only 30 data for train
NUM_EPOCH = 5
DETECT_SIZE = (608,608)
PRED_PABELS = ["Negative for Pneumonia","Typical Appearance","Indeterminate Appearance","Atypical Appearance"]

## Read DICM file

In [ ]:
import pydicom as dicom
import numpy as np
from PIL import Image
def dcm_to_pil(filename):
    scan = dicom.dcmread(filename)
    pil = None
    siz = (scan.Columns, scan.Rows)
    """
     if use pydicom some file read error.
      https://www.kaggle.com/c/siim-covid19-detection/discussion/239899
     so use jpeg-translated dataset
      https://www.kaggle.com/xhlulu/siim-covid19-resized-to-1024px-jpg
    """
    if filename.startswith("/kaggle/input/siim-covid19-detection/train/"):
        id = filename.split("/")[-1].split('.')[0]
        if os.path.isfile(f"/kaggle/input/siim-covid19-resized-to-1024px-jpg/train/{id}.jpg"):
            pil = Image.open(f"/kaggle/input/siim-covid19-resized-to-1024px-jpg/train/{id}.jpg")
    elif filename.startswith("/kaggle/input/siim-covid19-detection/test/"):
        id = filename.split("/")[-1].split('.')[0]
        if os.path.isfile(f"/kaggle/input/siim-covid19-resized-to-1024px-jpg/test/{id}.jpg"):
            pil = Image.open(f"/kaggle/input/siim-covid19-resized-to-1024px-jpg/test/{id}.jpg")
    if pil is None:
        pix = scan.pixel_array
        pix = (pix - np.min(pix)) / (np.max(pix) - np.min(pix))
        img = np.clip(np.round(pix * 0xff), 0, 0xff).astype(np.uint8)
        pil = Image.fromarray(img)
        siz = pil.size
    return pil.resize(DETECT_SIZE), siz

## Make Dataset Directory

In [ ]:
os.mkdir('/kaggle/working/yolov4/siim')

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")
df_train = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")
with open(f'/kaggle/working/yolov4/train.txt', 'w') as trainwf:
    for idx in tqdm(range(30 if RUN_TEST else len(df))):
        id = df.loc[idx].id.split('_')[0]
        fl = os.listdir(f'/kaggle/input/siim-covid19-detection/train/{id}')[0]
        fn = os.listdir(f'/kaggle/input/siim-covid19-detection/train/{id}/{fl}')[0]
        try:
            img, org_siz = dcm_to_pil(f'/kaggle/input/siim-covid19-detection/train/{id}/{fl}/{fn}')
        except:
            continue
        label = 0
        for i, l in enumerate(PRED_PABELS):
            if df.loc[idx][l] == 1:
                label = i
                break
        for j, box in enumerate(df_train[df_train.StudyInstanceUID==id].boxes):
            line = [f"{idx}_{j}.png"]
            if type(box) is not float:
                boxes = json.loads(box.replace("\'","\""))
                for b in boxes:
                    x = b['x'] / org_siz[0]
                    y = b['y'] / org_siz[1]
                    w = b['width'] / org_siz[0]
                    h = b['height'] / org_siz[1]
                    x1 = int(x*DETECT_SIZE[0])
                    x2 = x1 + int(w*DETECT_SIZE[0])
                    y1 = int(y*DETECT_SIZE[1])
                    y2 = y1 + int(h*DETECT_SIZE[1])
                    line.append(f"{x1},{y1},{x2},{y2},{label}")
            if len(line) > 1:
                img = img.convert("RGB")
                img.save(f'/kaggle/working/yolov4/siim/{idx}_{j}.png')
                trainwf.write(" ".join(line)+"\n")

  0%|          | 0/6054 [00:00<?, ?it/s]

# Training YoloV4

In [ ]:
from easydict import EasyDict

_BASE_DIR = '/kaggle/working/yolov4'

Cfg = EasyDict()

Cfg.use_darknet_cfg = True
Cfg.cfgfile = os.path.join(_BASE_DIR, 'cfg', 'yolov4.cfg')

Cfg.batch = 4
Cfg.subdivisions = 1
Cfg.width = DETECT_SIZE[0]
Cfg.height = DETECT_SIZE[1]
Cfg.channels = 3
Cfg.momentum = 0.949
Cfg.decay = 0.0005
Cfg.angle = 0
Cfg.saturation = 1.5
Cfg.exposure = 1.5
Cfg.hue = .1

Cfg.learning_rate = 0.00261
Cfg.burn_in = 1000
Cfg.max_batches = 500500
Cfg.steps = [400000, 450000]
Cfg.policy = Cfg.steps
Cfg.scales = .1, .1

Cfg.cutmix = 0
Cfg.mosaic = 1

Cfg.letter_box = 0
Cfg.jitter = 0.2
Cfg.classes = 80
Cfg.track = 0
Cfg.w = Cfg.width
Cfg.h = Cfg.height
Cfg.flip = 1
Cfg.blur = 0
Cfg.gaussian = 0
Cfg.boxes = 60  # box num
Cfg.TRAIN_EPOCHS = 1 if RUN_TEST else 30
Cfg.train_label = 'train.txt'
Cfg.val_label = 'train.txt'
Cfg.TRAIN_OPTIMIZER = 'adam'

if Cfg.mosaic and Cfg.cutmix:
    Cfg.mixup = 4
elif Cfg.cutmix:
    Cfg.mixup = 2
elif Cfg.mosaic:
    Cfg.mixup = 3

Cfg.checkpoints = os.path.join(_BASE_DIR, 'checkpoints')
Cfg.TRAIN_TENSORBOARD_DIR = os.path.join(_BASE_DIR, 'log')

Cfg.iou_type = 'iou'  # 'giou', 'diou', 'ciou'

Cfg.keep_checkpoint_max = 10

Cfg.dataset_dir = 'siim'
Cfg.gpu = "0"
Cfg.pretrained = "/kaggle/input/yolov4-predefend-weight/yolov4.conv.137.pth"

In [ ]:
import time
import logging
import os, sys, math
import argparse
from collections import deque
import datetime

import cv2
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.nn import functional as F
from tensorboardX import SummaryWriter
from easydict import EasyDict as edict

from dataset import Yolo_dataset
from models import Yolov4
from tool.darknet2pytorch import Darknet

from tool.tv_reference.utils import collate_fn as val_collate
from tool.tv_reference.coco_utils import convert_to_coco_api
from tool.tv_reference.coco_eval import CocoEvaluator

In [ ]:
from train import bboxes_iou,Yolo_loss, collate, init_logger, _get_date_str

In [ ]:
def train(model, device, config, epochs=5, batch_size=1, save_cp=True, log_step=20, img_scale=0.5):
    train_dataset = Yolo_dataset(config.train_label, config, train=True)

    n_train = len(train_dataset)

    train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                              num_workers=4, pin_memory=True, drop_last=True, collate_fn=collate)

    writer = SummaryWriter(log_dir=config.TRAIN_TENSORBOARD_DIR,
                           filename_suffix=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}',
                           comment=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}')
    # writer.add_images('legend',
    #                   torch.from_numpy(train_dataset.label2colorlegend2(cfg.DATA_CLASSES).transpose([2, 0, 1])).to(
    #                       device).unsqueeze(0))
    max_itr = config.TRAIN_EPOCHS * n_train
    # global_step = cfg.TRAIN_MINEPOCH * n_train
    global_step = 0
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {config.batch}
        Subdivisions:    {config.subdivisions}
        Learning rate:   {config.learning_rate}
        Training size:   {n_train}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images size:     {config.width}
        Optimizer:       {config.TRAIN_OPTIMIZER}
        Dataset classes: {config.classes}
        Train label path:{config.train_label}
        Pretrained:
    ''')

    # learning rate setup
    def burnin_schedule(i):
        if i < config.burn_in:
            factor = pow(i / config.burn_in, 4)
        elif i < config.steps[0]:
            factor = 1.0
        elif i < config.steps[1]:
            factor = 0.1
        else:
            factor = 0.01
        return factor

    if config.TRAIN_OPTIMIZER.lower() == 'adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=config.learning_rate / config.batch,
            betas=(0.9, 0.999),
            eps=1e-08,
        )
    elif config.TRAIN_OPTIMIZER.lower() == 'sgd':
        optimizer = optim.SGD(
            params=model.parameters(),
            lr=config.learning_rate / config.batch,
            momentum=config.momentum,
            weight_decay=config.decay,
        )
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, burnin_schedule)

    criterion = Yolo_loss(device=device, batch=config.batch // config.subdivisions, n_classes=config.classes)
    # scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=6, min_lr=1e-7)
    # scheduler = CosineAnnealingWarmRestarts(optimizer, 0.001, 1e-6, 20)

    save_prefix = 'Yolov4_epoch'
    saved_models = deque()
    model.train()
    for epoch in range(epochs):
        # model.train()
        epoch_loss = 0
        epoch_step = 0

        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img', ncols=50) as pbar:
            for i, batch in enumerate(train_loader):
                global_step += 1
                epoch_step += 1
                images = batch[0]
                bboxes = batch[1]

                images = images.to(device=device, dtype=torch.float32)
                bboxes = bboxes.to(device=device)

                bboxes_pred = model(images)
                loss, loss_xy, loss_wh, loss_obj, loss_cls, loss_l2 = criterion(bboxes_pred, bboxes)
                # loss = loss / config.subdivisions
                loss.backward()

                epoch_loss += loss.item()

                if global_step % config.subdivisions == 0:
                    optimizer.step()
                    scheduler.step()
                    model.zero_grad()

                if global_step % (log_step * config.subdivisions) == 0:
                    writer.add_scalar('train/Loss', loss.item(), global_step)
                    writer.add_scalar('train/loss_xy', loss_xy.item(), global_step)
                    writer.add_scalar('train/loss_wh', loss_wh.item(), global_step)
                    writer.add_scalar('train/loss_obj', loss_obj.item(), global_step)
                    writer.add_scalar('train/loss_cls', loss_cls.item(), global_step)
                    writer.add_scalar('train/loss_l2', loss_l2.item(), global_step)
                    writer.add_scalar('lr', scheduler.get_lr()[0] * config.batch, global_step)
                    pbar.set_postfix(**{'loss (batch)': loss.item(), 'loss_xy': loss_xy.item(),
                                        'loss_wh': loss_wh.item(),
                                        'loss_obj': loss_obj.item(),
                                        'loss_cls': loss_cls.item(),
                                        'loss_l2': loss_l2.item(),
                                        'lr': scheduler.get_lr()[0] * config.batch
                                        })
                    logging.debug('Train step_{}: loss : {},loss xy : {},loss wh : {},'
                                  'loss obj : {}，loss cls : {},loss l2 : {},lr : {}'
                                  .format(global_step, loss.item(), loss_xy.item(),
                                          loss_wh.item(), loss_obj.item(),
                                          loss_cls.item(), loss_l2.item(),
                                          scheduler.get_lr()[0] * config.batch))

                pbar.update(images.shape[0])

            if save_cp:
                try:
                    # os.mkdir(config.checkpoints)
                    os.makedirs(config.checkpoints, exist_ok=True)
                    logging.info('Created checkpoint directory')
                except OSError:
                    pass
                save_path = os.path.join(config.checkpoints, f'{save_prefix}{epoch + 1}.pth')
                torch.save(model.state_dict(), save_path)
                logging.info(f'Checkpoint {epoch + 1} saved !')
                saved_models.append(save_path)
                if len(saved_models) > config.keep_checkpoint_max > 0:
                    model_to_remove = saved_models.popleft()
                    try:
                        os.remove(model_to_remove)
                    except:
                        logging.info(f'failed to remove {model_to_remove}')

    writer.close()

In [ ]:
logging = init_logger(log_dir='log', stdout=False)
cfg = Cfg
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

model = Yolov4(cfg.pretrained, n_classes=cfg.classes)
model.to(device=device)

train(model=model,
      config=cfg,
      epochs=NUM_EPOCH,
      device=device, )

log file path:log/log_2021-05-20_09-43-46.txt


Epoch 1/5:   2%| | 76/4294 [00:34<28:06,  2.50img//opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Epoch 1/5: 100%|▉| 4292/4294 [28:41<00:00,  2.49im
Epoch 2/5: 100%|▉| 4292/4294 [28:39<00:00,  2.50im
Epoch 3/5: 100%|▉| 4292/4294 [28:42<00:00,  2.49im
Epoch 4/5: 100%|▉| 4292/4294 [28:43<00:00,  2.49im
Epoch 5/5: 100%|▉| 4292/4294 [28:51<00:00,  2.48im


# Prediction

In [ ]:
from models import Yolov4

In [ ]:
model = Yolov4(yolov4conv137weight=None, n_classes=int(cfg.classes), inference=True)
pretrained_dict = torch.load(f"checkpoints/Yolov4_epoch{NUM_EPOCH}.pth", map_location=device)
model.load_state_dict(pretrained_dict)
model.to(device)
model.eval()
""

''

In [ ]:
from tool.torch_utils import do_detect
from tool import utils
import cv2

## Threashold to Predict

In [ ]:
conf_thresh, nms_thresh = 0.4, 0.6
pred_name = ["negative", "typical", "indeterminate", "atypical"]

In [ ]:
def post_processing(img, conf_thresh, nms_thresh, output):

    # anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
    # num_anchors = 9
    # anchor_masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # strides = [8, 16, 32]
    # anchor_step = len(anchors) // num_anchors

    # [batch, num, 1, 4]
    box_array = output[0]
    # [batch, num, num_classes]
    confs = output[1]

    t1 = time.time()

    if type(box_array).__name__ != 'ndarray':
        box_array = box_array.cpu().detach().numpy()
        confs = confs.cpu().detach().numpy()

    num_classes = confs.shape[2]

    # [batch, num, 4]
    box_array = box_array[:, :, 0]

    # [batch, num, num_classes] --> [batch, num]
    max_conf = np.max(confs, axis=2)
    max_id = np.argmax(confs, axis=2)

    t2 = time.time()

    bboxes_batch = []
    for i in range(box_array.shape[0]):
       
        argwhere = max_conf[i] > conf_thresh
        l_box_array = box_array[i, argwhere, :]
        l_max_conf = max_conf[i, argwhere]
        l_max_id = max_id[i, argwhere]

        bboxes = []
        # nms for each class
        for j in range(num_classes):

            cls_argwhere = l_max_id == j
            ll_box_array = l_box_array[cls_argwhere, :]
            ll_max_conf = l_max_conf[cls_argwhere]
            ll_max_id = l_max_id[cls_argwhere]

            keep = utils.nms_cpu(ll_box_array, ll_max_conf, nms_thresh)
            
            if (keep.size > 0):
                ll_box_array = ll_box_array[keep, :]
                ll_max_conf = ll_max_conf[keep]
                ll_max_id = ll_max_id[keep]

                for k in range(ll_box_array.shape[0]):
                    bboxes.append([ll_box_array[k, 0], ll_box_array[k, 1], ll_box_array[k, 2], ll_box_array[k, 3], ll_max_conf[k], ll_max_conf[k], ll_max_id[k]])
        
        bboxes_batch.append(bboxes)

    t3 = time.time()

    return bboxes_batch

# Make Submission

In [ ]:
# make all id
study_id = []
image_id = []
study_id_file = []
image_id_file = []
for id in os.listdir(f'/kaggle/input/siim-covid19-detection/test'):
    study_id.append(id)
    sfile = None
    fl = os.listdir(f'/kaggle/input/siim-covid19-detection/test/{id}')
    for f in fl:
        fn = os.listdir(f'/kaggle/input/siim-covid19-detection/test/{id}/{f}')
        for d in fn:
            image_id.append(d.split('.')[0])
            image_id_file.append(f'/kaggle/input/siim-covid19-detection/test/{id}/{f}/{d}')
            if sfile is None:
                sfile = f'/kaggle/input/siim-covid19-detection/test/{id}/{f}/{d}'
    study_id_file.append(sfile)
    
pp = []
for sid, sfn in zip(study_id, study_id_file):
    pp.append((f'{sid}_study', sfn))
for mid, mfn in zip(image_id, image_id_file):
    pp.append((f'{mid}_image', mfn))

s = pd.read_csv("/kaggle/input/siim-covid19-detection/sample_submission.csv")
ids = s.id.values.tolist()
assert set(ids)==set([p[0] for p in pp]), "id miss"

In [ ]:
result_id = []
result_pred = []
chache_file = {}
for id, fn in tqdm(pp):
    img = None
    if fn in chache_file:
        dst = chache_file[fn]
    else:
        try:
            img, org_siz = dcm_to_pil(fn)
        except:
            dst = "negative 1 0 0 1 1"
    
    if img is not None:
        img = np.array(img.convert("RGB"))
        img = torch.from_numpy(img.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)
        output = model(img.to(device))
        boxes = post_processing(img, conf_thresh, nms_thresh, output)
        detect = [b for b in boxes[0] if b[6] < 4]
        if len(detect) == 0:
            dst = "negative 1 0 0 1 1"
        else:
            # Now, only One Result to Use
            detect = sorted(detect, key=lambda x:x[5])[-1]
            x1 = min(max(0,int(detect[0] * org_siz[0])), org_siz[0]-1)
            y1 = min(max(0,int(detect[1] * org_siz[1])), org_siz[1]-1)
            x2 = min(max(1,int(detect[2] * org_siz[0])), org_siz[0])
            y2 = min(max(1,int(detect[3] * org_siz[1])), org_siz[1])
            dst = f"{pred_name[detect[6]]} {detect[5]} {x1} {y1} {x2} {y2}"
        chache_file[fn] = dst

    result_id.append(id)
    result_pred.append(dst)

100%|██████████| 2477/2477 [05:30<00:00,  7.50it/s] 


In [ ]:
os.chdir('/kaggle/working')
pd.DataFrame({"id":result_id,"PredictionString":result_pred}).to_csv("submission.csv", index=False)

In [ ]:
!head submission.csv

id,PredictionString
2fb11712bc93_study,negative 1 0 0 1 1
19c66935e737_study,negative 1 0 0 1 1
2fc50bf199cd_study,negative 1 0 0 1 1
97c5d6eb413d_study,negative 1 0 0 1 1
593c3f815635_study,negative 1 0 0 1 1
f85b5d51e41d_study,negative 1 0 0 1 1
466f38a39d33_study,negative 1 0 0 1 1
2767827cedc2_study,negative 1 0 0 1 1
c85581436d7c_study,negative 1 0 0 1 1


In [ ]:
!rm -rf yolov4